
# 📚 Pipeline PRISMA + PICO con LLM (Scopus + WoS)
Este cuaderno te guía **paso a paso** para:
1) Cargar tu CSV (resultados Scopus/WoS).  
2) **Deduplicar** por DOI / Título normalizado.  
3) Calificar **cada registro** con un **LLM** usando **PICO** (Include/Exclude/Maybe + puntajes).  
4) Guardar resultados (CSV/Excel) y **filtrar al Top-10** más relevante para tu revisión.

> Puedes usar **DRY_RUN** para probar sin gastar tokens: genera salidas simuladas y no llama al LLM.


## 0) Requisitos

In [ ]:

# (Opcional) instala dependencias si hace falta en tu entorno
# !pip install -q pandas openai python-dotenv xlsxwriter


## 1) Configuración general (rutas y modo DRY_RUN)

In [43]:

import os, re, time, json
import pandas as pd

# === Paths de entrada / salida ===
INPUT_CSV = "../data/titles_result.csv"  # <-- Cambia aquí si tu archivo tiene otro nombre
OUT_CSV   = "../data/scored_results.csv"
OUT_XLSX  = "../data/scored_results.xlsx"

# === Modo simulación (no llama al LLM, útil para probar sin costo) ===
DRY_RUN = False  # <- pon True para simular (no requiere API key)

print(f"INPUT_CSV: {INPUT_CSV}\nOUT_CSV: {OUT_CSV}\nOUT_XLSX: {OUT_XLSX}\nDRY_RUN: {DRY_RUN}")


INPUT_CSV: ../data/titles_result.csv
OUT_CSV: ../data/scored_results.csv
OUT_XLSX: ../data/scored_results.xlsx
DRY_RUN: False


## 2) Define tu PICO y pesos (editable)

In [44]:
# === Research Context ===
RESEARCH_CONTEXT = {
    "main_question": "In industrial process valve control (P), do ML-based techniques (I) outperform traditional PID controllers (C) in loop performance, robustness, and autonomy (O)?",
    "sub_questions": [
        "When do they replace PID vs. assist it (hybrid approaches)?",
        "What validation do they report (simulation, pilot, field)?", 
        "How 'operable' are they in PLC/SCADA environments?"
    ],
    "background": "In process industries, PID controllers have been the predominant solution for regulating variables due to their simplicity and robustness. However, their performance is limited in complex conditions: multivariable systems (e.g., well testing separators controlling multiple valves in the same process), strongly nonlinear conditions with noise, disturbances, or changing dynamics require frequent re-tuning and may exceed their optimal capabilities. Therefore, interest has grown in control techniques based on neural networks and machine learning, which promise to adapt better to these complexities than traditional PIDs."
}

# === PICO (in English for better LLM comprehension) ===
PICO = {
    "P": "Industrial processes with valves (well testing, three-phase separation, refinery, and analogous systems).",
    "I": "ML-based control: neural networks (ANN/DL), reinforcement learning (RL), neuro-PID hybrids, agents/LLMs.",
    "C": "Traditional PID/PI controllers (baseline or reference).",
    "O": "Loop performance (overshoot, settling, IAE/ISE, RMS error), robustness, autonomy (less human intervention), PLC/SCADA feasibility."
}

# === Pesos de la rúbrica (si cambias la escala, ajusta aquí) ===
WEIGHTS = {
    "relevance_valve": 3,           # 0-3
    "ml_strength": 3,               # 0-3
    "vs_pid": 2,                    # 0-2
    "validation": 2,                # 0-2
    "industrial_similarity": 2,     # 0-2
    "peer_review": 1,               # 0-1
    "data_code": 1                  # 0-1
}

PICO

{'P': 'Industrial processes with valves (well testing, three-phase separation, refinery, and analogous systems).',
 'I': 'ML-based control: neural networks (ANN/DL), reinforcement learning (RL), neuro-PID hybrids, agents/LLMs.',
 'C': 'Traditional PID/PI controllers (baseline or reference).',
 'O': 'Loop performance (overshoot, settling, IAE/ISE, RMS error), robustness, autonomy (less human intervention), PLC/SCADA feasibility.'}

## 3) Prompt del LLM (con placeholders PICO)

In [45]:
# Updated prompt for PRISMA systematic review using ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """You are assisting a PRISMA-style systematic review on industrial valve control systems.

    RESEARCH CONTEXT:
    Main Question: {main_question}
    
    Sub-questions: 
    1. {sub_question_1}
    2. {sub_question_2} 
    3. {sub_question_3}
    
    Background: {background}

    PICO Framework:
    - P (Population/Problem): {P}
    - I (Intervention): {I}
    - C (Comparison): {C}
    - O (Outcomes): {O}

    EVALUATION RUBRIC (return integers only in the ranges indicated):
    - relevance_valve: 0–3 (direct valve control with closed-loop metrics=3; tangential valve mention=1; no valve=0)
    - ml_strength: 0–3 (RL/Deep NN/online learning=3; shallow/heuristic ML=1; no ML=0)
    - vs_pid: 0–2 (explicit quantitative comparison vs PID=2; mentions PID as baseline=1; no PID comparison=0)
    - validation: 0–2 (conceptual only=0; simulation studies=1; pilot/field experiments=2)
    - industrial_similarity: 0–2 (well testing/separation/ICV/choke=2; related process industry=1; academic/other=0)
    - peer_review: 0–1 (peer-reviewed journal/conference=1; preprint/thesis=0)
    - data_code: 0–1 (datasets/code available or reproducibility mentioned=1; else=0)

    RECORD TO EVALUATE:
    - Title: {title}
    - Source: {source_title}
    - DOI: {doi}
    - Keywords: {keywords}
    - Abstract: {abstract}
    
    Rate this record's relevance to our research question and provide your assessment.
    """
)

In [46]:
from pydantic import BaseModel, Field
from typing import Optional, Literal

class Scores(BaseModel):
    relevance_valve: int = Field(description="0-3: direct valve control with closed-loop metrics=3; tangential=0")
    ml_strength: int = Field(description="0-3: RL/Deep NN/online learning=3; shallow/heuristic ML=1; none=0")
    vs_pid: int = Field(description="0-2: explicit quantitative comparison vs PID=2; mentions PID as baseline=1; none=0")
    validation: int = Field(description="0-2: conceptual=0; simulation=1; pilot/field or bench experiments=2")
    industrial_similarity: int = Field(description="0-2: well testing/separation/ICV/choke=2; related process=1; other=0")
    peer_review: int = Field(description="0-1: 1 if peer-reviewed journal/conference; 0 otherwise/preprint")
    data_code: int = Field(description="0-1: 1 if datasets/code openly available or reproducibility noted; else 0")

class Notes(BaseModel):
    ml_technique: str = Field(description="short string describing ML technique used")
    control_task: str = Field(description="level/pressure/flow/valve positioning/other")
    domain: str = Field(description="well testing/ICV/choke/separation/other")
    metrics: str = Field(description="e.g., overshoot/settling/IAE/ISE/error/robustness")
    doi_confirmed: bool = Field(description="true if DOI is valid and confirmed")

class ScoredRecord(BaseModel):
    decision: Literal["Include", "Exclude", "Maybe"] = Field(description="Include/Exclude/Maybe decision")
    reason: str = Field(description="one-line reason for decision")
    scores: Scores = Field(description="scoring rubric results")
    notes: Notes = Field(description="additional analysis notes")

In [47]:
import sys
sys.path.append("..")  # Asegura que el directorio padre esté en el path
from model.llm import ModelFactory

model_core = "gpt4omini"

factory = ModelFactory(model_name=model_core, temperature=0)
llm = factory.create_model()

structured_llm = llm.with_structured_output(ScoredRecord)

chain = prompt | structured_llm

In [48]:
# Test con datos reales del CSV y nuevo contexto de investigación
inputs = {
    # Research context
    "main_question": RESEARCH_CONTEXT["main_question"],
    "sub_question_1": RESEARCH_CONTEXT["sub_questions"][0],
    "sub_question_2": RESEARCH_CONTEXT["sub_questions"][1], 
    "sub_question_3": RESEARCH_CONTEXT["sub_questions"][2],
    "background": RESEARCH_CONTEXT["background"],
    # PICO framework  
    "P": PICO["P"],
    "I": PICO["I"],
    "C": PICO["C"],
    "O": PICO["O"],
    # Record data
    "title": "Reinforcement learning for control of valves",
    "source_title": "Machine Learning with Applications",
    "doi": "10.1016/j.mlwa.2021.100030",
    "keywords": "",  # No keywords in this record
    "abstract": ""   # No abstract available in CSV
}

print("Testing enhanced chain with research context...")
print(f"Title: {inputs['title']}")
print(f"DOI: {inputs['doi']}")
print(f"Research Question: {inputs['main_question']}")
print("\nCalling LLM...")

results = chain.invoke(inputs)
print("\nResults:")
print(f"Decision: {results.decision}")
print(f"Reason: {results.reason}")
print(f"Scores: {results.scores}")
print(f"Notes: {results.notes}")

Testing enhanced chain with research context...
Title: Reinforcement learning for control of valves
DOI: 10.1016/j.mlwa.2021.100030
Research Question: In industrial process valve control (P), do ML-based techniques (I) outperform traditional PID controllers (C) in loop performance, robustness, and autonomy (O)?

Calling LLM...


2025-08-12 05:15:09,265 - _client - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Results:
Decision: Include
Reason: The study focuses on reinforcement learning for valve control, directly addressing the research question.
Scores: relevance_valve=3 ml_strength=3 vs_pid=2 validation=1 industrial_similarity=2 peer_review=1 data_code=1
Notes: ml_technique='Reinforcement Learning' control_task='Valve control' domain='Industrial processes' metrics='Loop performance, robustness, autonomy' doi_confirmed=True


## 5) Funciones auxiliares (dedup, normalización)

In [49]:

def normalize_title(t: str) -> str:
    if not isinstance(t, str):
        return ""
    t = t.lower().strip()
    t = re.sub(r"\s+", " ", t)
    t = re.sub(r"[^\w\s]", "", t)
    return t

def is_peer_reviewed(source_title: str) -> int:
    s = (source_title or "").lower()
    if any(k in s for k in ["journal", "transactions", "proceedings", "conference", "letters", "acta", "control", "engineering", "ifac", "ieee", "elsevier", "springer", "wiley", "mdpi", "acs"]):
        return 1
    return 0

def deduplicate(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in ["Title","Source title","DOI","Abstract","Author Keywords","Document Type","Publication Year","source"]:
        if c not in df.columns:
            df[c] = None
    df["title_norm"] = df["Title"].apply(normalize_title)
    df["_doi_norm"] = df["DOI"].astype(str).str.strip().str.lower()
    df["_peer"] = df["Source title"].apply(is_peer_reviewed)

    df = df.sort_values(["_doi_norm","_peer"], ascending=[True, False])
    df = df.drop_duplicates(subset=["_doi_norm","title_norm"], keep="first")

    df = df.sort_values(["title_norm","_peer"], ascending=[True, False])
    df = df.drop_duplicates(subset=["title_norm"], keep="first")

    return df.drop(columns=["_peer"])


In [50]:
def score_with_llm(df: pd.DataFrame, sleep_between_calls: float = 0.8) -> pd.DataFrame:
    """
    Califica cada registro del DataFrame usando la chain de LangChain con structured output.
    """
    rows = []
    
    print(f"Procesando {len(df)} registros...")
    
    for i, r in df.iterrows():
        print(f"Procesando registro {i+1}/{len(df)}: {r.get('Title', 'Sin título')[:50]}...")
        
        # Preparar input para la chain con contexto de investigación
        inputs = {
            # Research context
            "main_question": RESEARCH_CONTEXT["main_question"],
            "sub_question_1": RESEARCH_CONTEXT["sub_questions"][0],
            "sub_question_2": RESEARCH_CONTEXT["sub_questions"][1],
            "sub_question_3": RESEARCH_CONTEXT["sub_questions"][2],
            "background": RESEARCH_CONTEXT["background"],
            # PICO framework
            "P": PICO["P"],
            "I": PICO["I"], 
            "C": PICO["C"],
            "O": PICO["O"],
            # Record data
            "title": str(r.get("Title", "")),
            "source_title": str(r.get("Source title", "")),
            "doi": str(r.get("DOI", "")),
            "keywords": str(r.get("Author Keywords", "")),
            "abstract": str(r.get("Abstract", "")) if "Abstract" in df.columns else ""
        }
        
        try:
            # Llamar a la chain
            result = chain.invoke(inputs)
            
            # Calcular score total basado en WEIGHTS
            total_score = (
                result.scores.relevance_valve * WEIGHTS["relevance_valve"] +
                result.scores.ml_strength * WEIGHTS["ml_strength"] +
                result.scores.vs_pid * WEIGHTS["vs_pid"] +
                result.scores.validation * WEIGHTS["validation"] +
                result.scores.industrial_similarity * WEIGHTS["industrial_similarity"] +
                result.scores.peer_review * WEIGHTS["peer_review"] +
                result.scores.data_code * WEIGHTS["data_code"]
            )
            
            # Crear registro de salida
            out = {
                "llm_decision": result.decision,
                "llm_reason": result.reason.strip(),
                "score_relevance_valve": result.scores.relevance_valve,
                "score_ml_strength": result.scores.ml_strength,
                "score_vs_pid": result.scores.vs_pid,
                "score_validation": result.scores.validation,
                "score_industrial_similarity": result.scores.industrial_similarity,
                "score_peer_review": result.scores.peer_review,
                "score_data_code": result.scores.data_code,
                "score_overall": total_score,
                "ml_technique": result.notes.ml_technique,
                "control_task": result.notes.control_task,
                "domain": result.notes.domain,
                "metrics": result.notes.metrics,
                "doi_confirmed": result.notes.doi_confirmed
            }
            
        except Exception as e:
            print(f"Error procesando registro {i+1}: {e}")
            # Crear registro de error
            out = {
                "llm_decision": "Exclude",
                "llm_reason": f"Error de procesamiento: {str(e)}",
                "score_relevance_valve": 0,
                "score_ml_strength": 0,
                "score_vs_pid": 0,
                "score_validation": 0,
                "score_industrial_similarity": 0,
                "score_peer_review": 0,
                "score_data_code": 0,
                "score_overall": 0,
                "ml_technique": "",
                "control_task": "",
                "domain": "",
                "metrics": "",
                "doi_confirmed": False
            }
        
        rows.append(out)
        time.sleep(sleep_between_calls)
    
    # Combinar DataFrame original con nuevas columnas
    scored_df = pd.DataFrame(rows, index=df.index)
    result_df = pd.concat([df.reset_index(drop=True), scored_df.reset_index(drop=True)], axis=1)
    
    print("✅ Calificación completada!")
    return result_df

In [51]:
def shortlist_top10(scored: pd.DataFrame) -> pd.DataFrame:
    """
    Filtra y ordena el DataFrame para obtener el top-10 más relevante.
    Prioriza papers con control de válvulas, scores altos, y decisiones Include/Maybe.
    """
    # Filtros de relevancia
    has_valve_control = (
        scored["Title"].str.contains(r"valv", case=False, na=False) & 
        scored["Title"].str.contains(r"control", case=False, na=False)
    )
    
    strong_scores = scored["score_overall"] >= 7
    include_like = scored["llm_decision"].isin(["Include", "Maybe"])
    
    # Aplicar filtros
    prelim = scored[(has_valve_control | strong_scores) & include_like].copy()
    
    if len(prelim) == 0:
        print("⚠️  No se encontraron registros que cumplan los criterios. Mostrando top 10 por score general.")
        prelim = scored.copy()
    
    # Ordenar por prioridad: vs_pid > validation > score_overall
    prelim = prelim.sort_values(
        by=["score_vs_pid", "score_validation", "score_overall"],
        ascending=[False, False, False]
    )
    
    top10 = prelim.head(10)
    print(f"✅ Top-10 seleccionado de {len(prelim)} candidatos relevantes.")
    
    return top10

## 8) Ejecutar: cargar → deduplicar → calificar (LLM) → guardar

In [52]:
# Ejecutar: cargar → deduplicar → calificar (LLM) → guardar

print("🔄 Cargando datos...")
df = pd.read_csv(INPUT_CSV)
print(f"📄 Filas cargadas: {len(df)}")

print("\n🔄 Deduplicando registros...")
ddf = deduplicate(df)
print(f"📄 Tras deduplicar: {len(ddf)} registros únicos")

print("\n🔄 Calificando con LLM...")
scored = score_with_llm(ddf, sleep_between_calls=0.8)

print("\n🔄 Ordenando por score general...")
scored = scored.sort_values(by="score_overall", ascending=False)

print(f"\n💾 Guardando resultados completos...")
scored.to_csv(OUT_CSV, index=False)
print(f"✅ Guardado: {OUT_CSV}")

# Mostrar resumen de resultados
print(f"\n📊 Resumen de calificaciones:")
print(f"  - Include: {len(scored[scored['llm_decision'] == 'Include'])}")
print(f"  - Maybe: {len(scored[scored['llm_decision'] == 'Maybe'])}")
print(f"  - Exclude: {len(scored[scored['llm_decision'] == 'Exclude'])}")
print(f"  - Score promedio: {scored['score_overall'].mean():.1f}")
print(f"  - Score máximo: {scored['score_overall'].max()}")

scored.head()

🔄 Cargando datos...
📄 Filas cargadas: 79

🔄 Deduplicando registros...
📄 Tras deduplicar: 57 registros únicos

🔄 Calificando con LLM...
Procesando 57 registros...
Procesando registro 51/57: A Literature Review of the Positive Displacement C...


2025-08-12 05:15:32,696 - _client - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Procesando registro 58/57: A Method for Detecting Key Points of Transferring ...


2025-08-12 05:15:36,073 - _client - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Procesando registro 57/57: A novel modelling of glue allowance prediction for...


...

2025-08-12 05:18:54,035 - _client - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Procesando registro 62/57: Utilizing Selected Machine Learning Methods for Co...


2025-08-12 05:18:58,709 - _client - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✅ Calificación completada!

🔄 Ordenando por score general...

💾 Guardando resultados completos...
✅ Guardado: ../data/scored_results.csv

📊 Resumen de calificaciones:
  - Include: 1
  - Maybe: 0
  - Exclude: 56
  - Score promedio: 2.2
  - Score máximo: 29


,source,Title,Author Keywords,Source title,DOI,Abstract,Document Type,Publication Year,title_norm,_doi_norm,...,score_validation,score_industrial_similarity,score_peer_review,score_data_code,score_overall,ml_technique,control_task,domain,metrics,doi_confirmed
46,WoS,Reinforcement learning for control of valves,NaN,MACHINE LEARNING WITH APPLICATIONS,10.1016/j.mlwa.2021.100030,None,None,None,reinforcement learning for control of valves,10.1016/j.mlwa.2021.100030,...,1,2,1,0,29,Reinforcement Learning,Valve control,Industrial processes,"Loop performance, robustness",True
31,scopus,Fault Detection of Flow Control Valves Using O...,flow control valve; LightGBM; online fault det...,Actuators,10.3390/act13060222,None,None,None,fault detection of flow control valves using o...,10.3390/act13060222,...,0,0,1,0,10,LightGBM,fault detection,flow control,not applicable,True
32,scopus,Implementation of ANN-Based Auto-Adjustable fo...,control; embedded; FPGA; neural network; neuro...,Micromachines,10.3390/mi13060890,None,None,None,implementation of annbased autoadjustable for ...,10.3390/mi13060890,...,0,0,1,0,10,ANN-based control,pneumatic servo system,robotics,not specified,True
16,scopus,Control valve stiction detection using Markov ...,convolutional neural network; machine learning...,Canadian Journal of Chemical Engineering,10.1002/cjce.25054,None,None,None,control valve stiction detection using markov ...,10.1002/cjce.25054,...,0,0,1,0,10,Deep Convolutional Neural Network,Stiction detection,Other,Not applicable,True
11,scopus,Automatic oscillations detection and classific...,control valve; machine learning; model free; O...,Transactions of the Institute of Measurement a...,10.1177/01423312221118129,None,None,None,automatic oscillations detection and classific...,10.1177/01423312221118129,...,0,0,1,0,7,generalized machine learning algorithms,oscillation detection,control systems,oscillation detection accuracy,True


## 9) Filtrado al Top-10 (prioriza vs PID, validación y puntaje total)

In [53]:
with pd.ExcelWriter(OUT_XLSX, engine="xlsxwriter") as w:
    scored.to_excel(w, sheet_name="All_Scored", index=False)

print("Guardado:")
print("-", OUT_CSV)
print("-", OUT_XLSX)

Guardado:
- ../data/scored_results.csv
- ../data/scored_results.xlsx


In [54]:
# Generar Top-10 filtrado y optimizado
print("🔄 Generando Top-10...")
top10 = shortlist_top10(scored)

# Guardar Top-10
top10.to_excel("../data/Top10.xlsx", index=False)
top10.to_csv("../data/Top10.csv", index=False)
print(f"💾 Top-10 guardado: Top10.xlsx y Top10.csv")

# Mostrar resumen del Top-10
print(f"\n🏆 TOP-10 MÁS RELEVANTES:")
display_cols = ["Title", "DOI", "score_overall", "llm_decision", "llm_reason"]
top10_display = top10[display_cols].copy()
top10_display["Title"] = top10_display["Title"].str[:50] + "..."  # Truncar títulos largos

print(top10_display.to_string(index=False))

🔄 Generando Top-10...
✅ Top-10 seleccionado de 1 candidatos relevantes.
💾 Top-10 guardado: Top10.xlsx y Top10.csv

🏆 TOP-10 MÁS RELEVANTES:
                                          Title                        DOI  score_overall llm_decision                                                                                             llm_reason
Reinforcement learning for control of valves... 10.1016/j.mlwa.2021.100030             29      Include The study focuses on reinforcement learning for valve control, which aligns with the research context.



## 10) Siguientes pasos
- Revisa `Top10.xlsx` y el `sheet All_Scored` para ajustar criterios o PICO.
- Si deseas un **núcleo más estricto** (solo control de válvula con comparación vs PID), filtra por:
  - `Title` contenga *"valv"* y *"control"* y *"PID"*
  - `score_vs_pid >= 1` y `score_relevance_valve >= 2`
- Puedes cambiar el **PROMPT_TEMPLATE** o los **pesos** y re-ejecutar.
- Exporta los incluidos a **Zotero** (CSV/BetterBibTeX) y a **Rayyan** para PRISMA.
